<a href="https://colab.research.google.com/github/9-coding/PyTorch/blob/main/Lecture-AI_programming/week09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week09

torch.nn.Module 객체의 state_dict() 메서드는

- 모델의 학습 가능한 매개변수(가중치와 바이어스)의 상태와
- 버퍼(예: BatchNorm의 running mean과 variance 등)의 상태를 저장하는
- collections.OrderedDict 객체를 반환.
- 반환된 객체는 모델의 현재 상태를 나타내며, 저장 및 로드가 가능함.

Serialization: 직렬화로 저장.
개체 직렬화
key: value 형태로

1. OrderedDict 형태:
- state_dict()는 attribute 이름을 키로 하고,
- 그에 대응하는 torch.Tensor를 값으로 갖는
- collections.OrderedDict 객체를 반환.
2. 학습 가능한 매개변수:
- state_dict()는 torch.nn.Parameter 객체로 정의된 모든 학습 가능한 attributes를 포함.
3. 버퍼:
- 모델에 포함된 모든 버퍼(예: BatchNorm 계층의 running mean과 variance)도 포함.

OrderedDict: 순서가 정의된 딕셔너리 형태.

In [1]:
import torch
import torch.nn as nn
from collections import OrderedDict

# 간단한 모델 정의
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.linear = nn.Linear(10, 5)
        self.bn = nn.BatchNorm1d(5)
        # self.sub = nn.Sequential(
        #     OrderedDict({
        #         'ds00_layer': nn.Linear(5,5),
        #         'ds00_bn':nn.BatchNorm1d(5),
        #         'ds00_act': nn.ReLU(),
        #         'ds01_layer': nn.Linear(5,5),
        #     })
        # )

    def forward(self, x):
        x = self.linear(x)
        x = self.bn(x)
        return x

# 모델 인스턴스 생성
model = MyModel()

# 모델의 state_dict 가져오기
state_dict = model.state_dict()

# state_dict 내용 출력
for key, value in state_dict.items():
    print(f"{key}: {value.shape}")

linear.weight: torch.Size([5, 10])
linear.bias: torch.Size([5])
bn.weight: torch.Size([5])
bn.bias: torch.Size([5])
bn.running_mean: torch.Size([5])
bn.running_var: torch.Size([5])
bn.num_batches_tracked: torch.Size([])


In [2]:
# 모델의 state_dict 가져오기
state_dict0 = model.state_dict(prefix='ds', keep_vars=True)

# state_dict 내용 출력
for key, value in state_dict0.items():
    print(f"{key}: {value.shape}")

dslinear.weight: torch.Size([5, 10])
dslinear.bias: torch.Size([5])
dsbn.weight: torch.Size([5])
dsbn.bias: torch.Size([5])
dsbn.running_mean: torch.Size([5])
dsbn.running_var: torch.Size([5])
dsbn.num_batches_tracked: torch.Size([])


메서드 parameters: state_dict(desitnation=None, prefix='', keep_vars=False)
keep_vars 는 기본값이 False로 buffers와 parameters 의 값만을 추출할지를 결정
keep_vars=True 인 경우, 값 대신 tensor객체로 데이터 버퍼를 가지고 있는 dictionary가 반환됨.
value 가 파라메터인 경우엔 Parameter 로 얻어지고,
value 가 버퍼인 경우엔 Tensor 로 얻어짐.
keep_vars=True 인 경우, 메모리 사용량이 커지고, 매우 느리고 복잡한 동작이 이루어지지만. 다음의 장점을 가짐.
모델 디버깅: 모델 상태를 조사하고 특정 매개 변수나 버퍼의 값을 변경해야 하는 경우 유용
모델 커스터마이징: 모델을 불러온 후 특정 매개 변수나 버퍼의 값을 변경해야 하는 경우 유용
모델 저장 및 불러오기 확장: 모델 저장 및 불러오기 프로세스를 확장하고 추가적인 정보를 저장해야 하는 경우 유용
하지만, PyTorch의 버전이 정확히 맞아야만 동작할 수 있는 등의 제한점을 가짐.
저장의 용도로는 keep_vars=False를 사용하는 게 좋음.


linear.weight: nn.Linear 계층의 가중치 파라미터
linear.bias: nn.Linear 계층의 바이어스 파라미터
bn.weight: nn.BatchNorm1d 계층의 가중치 파라미터
bn.bias: nn.BatchNorm1d 계층의 바이어스 파라미터
bn.running_mean: nn.BatchNorm1d 계층의 running mean 버퍼
bn.running_var: nn.BatchNorm1d 계층의 running variance 버퍼
bn.num_batches_tracked: nn.BatchNorm1d 계층에서 배치의 수를 추적하는 버퍼

모델 저장

state_dict를 파일에 저장하여 나중에 모델을 복원할 수 있음.

In [ ]:

torch.save(model.state_dict(), 'model_state.pth')

모델 로드:

저장된 state_dict를 로드하여 모델을 복원.

In [ ]:
model = MyModel()
model.load_state_dict(torch.load('model_state.pth'))
model.eval() # 평가 모드로 전환 (선택 사항)`

파라미터 업데이트:

state_dict를 사용하여 모델의 특정 파라미터를 업데이트할 수 있음.

In [ ]:
state_dict['linear.weight'] = torch.ones_like(state_dict['linear.weight'])
model.load_state_dict(state_dict)

# Torch: Save and Load Model
PyTorch에서 model을 저장하는 방법은 크게 두 가지임.

모델의 Parameters (= weights and bias)를 저장 (Structure 등은 저장되지 않음).
model 전체를 저장하는 방법 (Parameters와 Structure 함께)
일반적으로 권장되는 방법은 1번임.

1번의 경우,

비록 모델의 구조를 정의하고 있는 class 의 instance 코드 상에서 생성하고,
이 instance로 로딩을 수행해줘야 하지만,
해당 class의 소스를 정확히 가지고 있을 경우 PyTorch 버전 등에 상관없이
이전과 동일한 모델을 load를 통해 얻을 수 있음.
위에서 정확히 가지고 있다라는 애기는 save할 때와 load할 때의 모델 클래스의 definition이 동일해야함을 의미함.

https://ds31x.tistory.com/263

In [3]:
# 필요한 library와 모듈 import
import torch
from torch import nn
from torch.nn import init
from collections import OrderedDict

# 간단한 linear regression model 정의.
# SimpleModel0와 SimpleModel1은
# 똑같은 구조이나 파라메터들의 초기값만 다름.
class SimpleModel0(nn.Module):

  def __init__(self, n_in_f, n_out_f):

    super().__init__()


    init_weigths = torch.ones( (n_in_f, n_out_f) )
    init_bias = torch.zeros( (n_out_f,) )

    self.l0 = nn.Linear(n_in_f, n_out_f)

    const_weight = 1.
    const_bias = 0.5

    init.constant_(self.l0.weight, const_weight)
    if self.l0.bias is not None:
      init.constant_(self.l0.bias, const_bias)

  def forward(self, x):
    return self.l0(x)

class SimpleModel1(nn.Module):

  def __init__(self, n_in_f, n_out_f):

    super().__init__()

    init_weigths = torch.ones( (n_in_f, n_out_f) )
    init_bias = torch.zeros( (n_out_f,) )

    self.l0 = nn.Linear(n_in_f, n_out_f)

    const_weight = 2.
    const_bias = 1.5

    init.constant_(self.l0.weight, const_weight)
    if self.l0.bias is not None:
      init.constant_(self.l0.bias, const_bias)

  def forward(self, x):
    return self.l0(x)


# 모델 객체를 생성하고, 이에 대한 파라메터 확인후
# 파라메터만 저장.
model = SimpleModel0(3,1)
print(list(model.named_parameters()))
torch.save(model.state_dict(), 'model_params.pth')

# 새로운 모델 객체를 생성.
# 해당 모델 객체는 구조는 같으나, 파라메터들의 초기값은 다름.
n_model = SimpleModel1(3,1)

print('===============')
for old, new in zip(model.parameters(), n_model.parameters()):

  if not torch.equal(old,new):
    print('model and n_model w/ default init do not have parameters with the same values!')
    break
else:
  print('model and n_model w/ default init have parameters with the same values!')
print('===============')

# 이전 저장한 parameters에 대한 state_dict를
# 로드하고 해당 state_dict로 새로만든 모델의
# 파라메터를 설정하고 이전 모델과 비교.
# load parameters and restore old parameters into new model
loaded_params_ordered_dict = torch.load('model_params.pth')
print(f'{type(loaded_params_ordered_dict)=}') # collections.OredredDict

ret_v = n_model.load_state_dict(loaded_params_ordered_dict)
print(f'{type(ret_v)}: {ret_v}')

print('===============')
for old, new in zip(model.parameters(), n_model.parameters()):

  if not torch.equal(old,new):
    print('model and n_model do not have parameters with the same values!')
    break
else:
  print('model and n_model have parameters with the same values!')

[('l0.weight', Parameter containing:
tensor([[1., 1., 1.]], requires_grad=True)), ('l0.bias', Parameter containing:
tensor([0.5000], requires_grad=True))]
model and n_model w/ default init do not have parameters with the same values!
type(loaded_params_ordered_dict)=<class 'collections.OrderedDict'>
<class 'torch.nn.modules.module._IncompatibleKeys'>: <All keys matched successfully>
model and n_model have parameters with the same values!


버전이 크게 바뀔 경우 안 될 확률이 매우 높음.
주기적으로 읽어서 최신 버전으로 다시 저장 필요.

# Tensor 비교

PyTorch에서 nn.Parameter 또는 tensor 객체 두 개가 같은 값을 가지는지 확인하는 방법은 텐서의 모든 요소가 동일한지 확인하는 것임.



이를 위해 제공되는 다음과 같은 함수 2개가 존재함.

torch.equal : 두 텐서의 모든 요소가 동일한지 확인
torch.allclose : 지정된 허용 오차 내에서 두 텐서가 거의 동일한지 확인.

## `torch.equal` 사용
https://ds31x.tistory.com/265

In [ ]:
import torch
import torch.nn as nn

# nn.Parameter 객체 생성
param1 = nn.Parameter(torch.tensor([1.0, 2.0, 3.0]))
param2 = nn.Parameter(torch.tensor([1.0, 2.0, 3.0]))
param3 = nn.Parameter(torch.tensor([1.0, 2.0, 4.0]))

# 값 비교
print(torch.equal(param1, param2))  # True
print(torch.equal(param1, param3))  # False

## `torch.allclose` 사용
이 방법은 두 텐서가 지정된 허용 오차 내에서 거의 동일한지를 확인.

이는 부동 소수점 연산의 미세한 차이로 인한 불일치를 허용할 수 있음 (권장).

https://ds31x.tistory.com/266

이름을 지정하면 디버깅시 확인이 매우 편함.

일반 list / dict를 사용하면 선언이 되지 않고 parameter로 선언이 안 되기 때문에 하습 자체가 불가.

loss가 기존보다 줄었다면 모델 저장 기능 추가. -> checkpoint.
epoch마다 저장하는 것은 파일명에 에포크수가 포함되어 있어야 함.
+날짜 및 시간

optimizer의 상태도 그대로 저장해야 함.